## Introduction

This notebook is made to synthetize the code of the kaggle competition hosted for the MLNS course. The competition was about performing link prediction on an actor network where nodes represent actors and edges between two nodes stand for co-occurrence on the same Wikipedia page.

## Utilitary Functions

Here are a list of functions that will remain relevant throughout all the notebook, that were used to help reading inputs and make submissions.

Along those functions are given necessary installs so that there are no issues in running the notebook.

In [ ]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import networkx as nx 
import csv
import torch
import pandas as pd 
import numpy as np

from typing import List, Dict

def load_set(train=False) -> List[str]:
    if train: 
        with open("train.txt", "r") as f:
            reader = csv.reader(f)
            dataset = list(reader)
        dataset = [element[0].split(" ") for element in dataset]
    else:
        with open("test.txt", "r") as f:
            reader = csv.reader(f)
            dataset = list(reader)
        dataset = [element[0].split(" ") for element in dataset]
    return dataset

def set_to_nx(dataset:List[str]) -> nx.Graph:
    g = nx.Graph()
    for elem in dataset:
        g.add_node(elem[0])
        g.add_node(elem[1])
        if elem[2] == "1":
            g.add_edge(elem[0], elem[1])
    return g 

def set_to_directed_nx(dataset:List[str]) -> nx.Graph:
    g = nx.DiGraph()
    for elem in dataset:
        g.add_node(elem[0])
        g.add_node(elem[1])
        if elem[2] == "1":
            g.add_edge(elem[0], elem[1])
    return g 

def get_non_edges(dataset:List[str]) -> List[str]:
    non_edge_list = []
    for elem in dataset:
        if elem[2] == "0":
            non_edge_list.append([elem[0], elem[1]])
    return non_edge_list

def info_to_dict() -> Dict[str, torch.Tensor]:
    node_information = pd.read_csv("node_information.csv", sep=",", header=None)
    node_names = node_information[node_information.columns[0]]
    node_information.drop(node_information.columns[0], axis=1, inplace=True)
    node_data = torch.from_numpy(node_information.to_numpy()).to(torch.float32)
    node_dict = {}
    for i, idx in enumerate(node_names):
        node_dict[str(idx)] = node_data[i].to_sparse()
    return node_dict

def generate_samples(graph:nx.Graph, non_edges:List[str], train_set_ratio:float):
    #function taken from the MLNS practical sessions
    if nx.is_connected(graph) is not True:
        raise ValueError("The graph contains more than one connected component!")
    residual_g = graph.copy()
    valid_pos_samples = []
    edges = list(residual_g.edges())
    np.random.shuffle(edges)
    valid_set_size = int((1.0 - train_set_ratio) * graph.number_of_edges())
    train_set_size = graph.number_of_edges() - valid_set_size
    num_of_pos_valid_samples = 0
    for edge in edges:
        residual_g.remove_edge(edge[0], edge[1])
        if nx.is_connected(residual_g):
            num_of_pos_valid_samples += 1
            valid_pos_samples.append(edge)
        else: 
            residual_g.add_edge(edge[0], edge[1])
        if num_of_pos_valid_samples == valid_set_size:
            break
    if num_of_pos_valid_samples != valid_set_size:
        raise ValueError("Enough positive edge samples could not be found!")
    train_pos_samples = list(residual_g.edges())
    np.random.shuffle(non_edges)
    train_neg_samples = non_edges[:train_set_size] 
    valid_neg_samples = non_edges[train_set_size:train_set_size + valid_set_size]
    train_samples = train_pos_samples + train_neg_samples
    train_labels = [1 for _ in train_pos_samples] + [0 for _ in train_neg_samples]
    # For valid set
    valid_samples = valid_pos_samples + valid_neg_samples
    valid_labels = [1 for _ in valid_pos_samples] + [0 for _ in valid_neg_samples]
    return residual_g, train_samples, train_labels, valid_samples, valid_labels

def create_submission(n_test:int, predictions:np.ndarray, pred_name:str):
    submission = zip(np.arange(n_test), predictions)
    # note: Kaggle requires that you add "ID" and "category" column headers
    csv_name = pred_name+".csv"
    with open(csv_name,"w") as pred:
        csv_out = csv.writer(pred)
        csv_out.writerow(i for i in ["ID", "Predicted"])
        for row in submission:
            csv_out.writerow(row)
        pred.close()

## Use of Supervised Learning on Graph Features

### Feature Extraction

The first approach tried was a traditional machine learning one, using global features of the graph extracted through networkx algorithms. Here are all the node features we use: 
- Degree centrality
- Betweenness centrality
- Pagerank
- HITS
- Katz Centrality

As those features are node-focused and we want to keep on the fact that prediction should be done symetrically due to the graph being undirected (the classifier should predict (a, b) whether we feed it (b, a) or (a, b) as an edge), we take as features the average and quadratic distance between source and target node features.

In addition, we add one global edge feature: Simrank.

All those features are normalized using a MinMaxScaler. 

In [ ]:
import tqdm
import networkx as nx 
import numpy as np

from sklearn.preprocessing import MinMaxScaler

class FeatureExtractor:
    def __init__(self, scaler=MinMaxScaler()) -> None:
        self.scaler = scaler

    def feature_extract(self, graph, samples, train):
        """
        Creates a feature vector for each edge of the graph contained in samples 
        """
        feature_vector = [] 
        
        deg_centrality = nx.degree_centrality(graph)
        betweeness_centrality = nx.betweenness_centrality(graph)
        p_rank = nx.pagerank(graph)
        hits = nx.hits(graph)
        hubs = hits[0]
        auth = hits[1]
        simrank = nx.simrank_similarity(graph)
        phi = np.abs(max(nx.adjacency_spectrum(graph)))
        alpha = (1 / phi - 0.01)
        katz = nx.katz_centrality(graph, alpha)

        for edge in tqdm.tqdm(samples):
            source_node, target_node = edge[0], edge[1]
            
            ## Global FEATURES
            # Degree Centrality
            src_dg_ctrl = deg_centrality[source_node]
            tgt_dg_ctrl = deg_centrality[target_node]
            dg_avg = (src_dg_ctrl+tgt_dg_ctrl)/2 
            dg_std = (src_dg_ctrl-tgt_dg_ctrl)**2
            
            src_btw_ctrl = betweeness_centrality[source_node]
            tgt_btw_ctrl = betweeness_centrality[target_node]
            btw_avg = (src_btw_ctrl+tgt_btw_ctrl)/2 
            btw_std = (src_btw_ctrl-tgt_btw_ctrl)**2

            #Auth
            src_hubs, tgt_hubs = hubs[source_node], hubs[target_node]
            hubs_avg, hubs_std = (src_hubs+tgt_hubs)/2, (src_hubs - tgt_hubs)**2
            src_auth, tgt_auth = auth[source_node], auth[target_node]
            auth_avg, auth_std = (src_auth+tgt_auth)/2, (src_auth - tgt_auth)**2

            #prank
            src_p_rank = p_rank[source_node]
            tgt_p_rank = p_rank[target_node]
            p_rank_avg = (src_p_rank + tgt_p_rank)/2
            p_rank_std = (src_p_rank - tgt_p_rank)**2

            #katz 
            src_katz = katz[source_node]
            tgt_katz = katz[target_node]
            katz_avg = (src_katz + tgt_katz)/2 
            katz_std = (src_katz - tgt_katz)**2

            #simrank 
            edge_simrank = simrank[source_node][target_node]

            #features = np.concatenate((vect_features, emb_features))
            features = np.array([dg_avg, dg_std, btw_avg, btw_std, hubs_avg, hubs_std, auth_avg, auth_std, p_rank_avg, p_rank_std, katz_avg, katz_std, edge_simrank])
            feature_vector.append(features) 
        feature_vector = np.array(feature_vector)
        if train:
            feature_vector = self.scaler.fit_transform(feature_vector)
        else:
            feature_vector = self.scaler.transform(feature_vector)
        return feature_vector

### Classifier

Once those features are extracted, a Logistic Regression is performed to realize the classification tasks. Parameters (C, L1_ratio, penalty, solver) were found using Grid Search Cross Validation.

Other algorithms that were tried were RandomForest and XGBoost, but in practice, they performed less better than LogisticRegression in our use case.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

train_set = load_set(train=True)
g = set_to_nx(train_set)
non_edges = get_non_edges(train_set)
extractor = FeatureExtractor()
clf = LogisticRegression(C=100, l1_ratio=0.1, penalty="elasticnet", solver="saga", max_iter=10000)
evaluate_method = False
if evaluate_method:
        residual_g, train_samples, train_labels, valid_samples, valid_labels = generate_samples(g, non_edges, 0.8)
        train_features = extractor.feature_extract(residual_g, train_samples, train=True)
        valid_features = extractor.feature_extract(residual_g, valid_samples, train=False)
        param_dict = {}
        param_dict["penalty"] = ["elasticnet"]
        param_dict["solver"] = ["saga"]
        param_dict["max_iter"] = [10000]
        param_dict["C"] = [0.1, 1, 10, 100, 1000]
        param_dict["l1_ratio"] = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
        clf = GridSearchCV(clf, param_dict, verbose=True)
        clf.fit(train_features, train_labels)
        print(clf.score(valid_features, valid_labels))
        print(clf.best_params_)   
else:
        total_samples = list(g.edges) + non_edges 
        total_labels = [1 for _ in range(len(g.edges))] + [0 for _ in range(len(non_edges))]
        total_features = extractor.feature_extract(g, total_samples, train=True)
        clf.fit(total_features, total_labels)
        test_set = load_set(train=False)
        n_test = len(test_set)
        test_features = extractor.feature_extract(g, test_set, train=False) 
        pred = clf.predict(test_features)
        create_submission(n_test, pred, pred_name="submissions/supervised_pred")

100%|██████████| 3498/3498 [00:00<00:00, 79707.48it/s]


## Prediction using Edge Features Ranking

### Feature Extraction

Another approach that was tried was an unsupervised one, using Edge Features that are commonly used for Link Prediction as a way to rank the more likely edges in the test set to be connected.

The features that were used are the following:
- Common Neighbor Centrality
- Preferential Attachment
- Jaccard Coefficient
- Ressource Allocation Index
- Adamic Adar Index
- Shortest Path Length between target node and source node

At first, those features were normalized, but we noticed that it was not necessary to do so, as we only cared about how each edge was ranked relatively to each feature.

In [ ]:
import tqdm
import networkx as nx 
import numpy as np

from sklearn.preprocessing import MinMaxScaler

class FeatureExtractor:
    def __init__(self, scaler=MinMaxScaler()) -> None:
        self.scaler = scaler

    def feature_extract(self, graph, samples, train):
        """
        Creates a feature vector for each edge of the graph contained in samples 
        """
        feature_vector = [] 
        epured_samples = []
        for edge in samples:
            if edge[0] != edge[1]:
                epured_samples.append(edge)
        cnc = nx.common_neighbor_centrality(graph, epured_samples) 
        cnc_list = list(cnc)   
        for edge in tqdm.tqdm(samples):
            source_node, target_node = edge[0], edge[1]

            ## EDGE FEATURES
            # Preferential Attachement 
            pref_attach = list(nx.preferential_attachment(graph, [(source_node, target_node)]))[0][2]
            # Jaccard
            jaccard_coeff = list(nx.jaccard_coefficient(graph, [(source_node, target_node)]))[0][2]
            # Ressource Allocation Index
            rai = list(nx.resource_allocation_index(graph, [(source_node, target_node)]))[0][2]
            # Adamic Adar Index 
            aai = 0 
            if target_node != source_node: 
                aai = list(nx.adamic_adar_index(graph, [(source_node, target_node)]))[0][2]
            # Common Neighbor Centrality
            cnc = 0
            if target_node != source_node:
                for elem in cnc_list: 
                    if elem[0] == source_node and elem[1] == target_node:
                        cnc = elem[2] 
                        continue
            # Shortest Path
            sp = nx.shortest_path_length(graph, source_node, target_node)

            features = np.array([jaccard_coeff, rai, pref_attach, aai, cnc, sp])
            feature_vector.append(features) 
        feature_vector = np.array(feature_vector)
        return feature_vector
        

### Prediction

Under the assumption that there are as many edges as non-edges in the test set (which is usually a strong assumption, but seems to be the case for this competition !), we take the half of the edges with the better average rank.

As actually a lot of edges are between nodes that do not have common neighbors, Adamic Adar, RAI and Jaccard are often equal to 0. Thus, in our ranking, the final submission used the "min" method of pandas in order to bring out less those nodes compared to those with an AA, RAI and Jaccard that are strictly positive.

In [ ]:
import pandas as pd 
import numpy as np 

train_set = load_set(train=True)
g = set_to_nx(train_set)
extractor = FeatureExtractor()
test_set = load_set(train=False)
n_test = len(test_set)
test_features = extractor.feature_extract(g, test_set, True)
test_features = pd.DataFrame(test_features)
test_ranks = test_features.rank(axis=0, method="min")
test_features = np.mean(test_ranks.to_numpy(), axis=1)
sorted_features = np.argsort(test_features)[::-1]
pred = np.zeros(n_test)
for i in range(int(len(sorted_features)*0.5)):
    idx = sorted_features[i]
    pred[idx] = 1
create_submission(n_test, pred, pred_name="unsupervised_pred")

100%|██████████| 3498/3498 [00:01<00:00, 2547.75it/s]


## Graph Neural Network to make prediction based on Node Information

### Idea

Node Information was a tough feature to exploit. It was unusable for both previous types of predictors, as it often made predictions worse if it was put alongside the other features.

The idea we had in the end was using a Graph Neural Network to extract embeddings from the Node features using the Graph structure that was provided, and then make prediction using an MLP based on those features.

Two things have to be noted here:
- As DGL turns any undirected graph into a directed graph, we had to remove edges that were in double in order to not bias the training process.
- As we want to keep symmetry between predictions (results should not change for (a, b) and (b, a)) we feed to the MLP predictor the average and quadratic distance between source and target embeddings output by the Graph Neural Network.

The Neural Network that was used here was GraphSage.

### Networks

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

from dgl.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, output_feats, n_layers, dropout, skip=False):
        super(GraphSAGE, self).__init__()
        self.convs = nn.ModuleList()
        first_conv = SAGEConv(in_feats, h_feats, 'mean')
        self.convs.append(first_conv)
        self.n_layers = n_layers
        for i in range(n_layers - 2):
            add_conv = SAGEConv(h_feats, h_feats, 'mean')
            self.convs.append(add_conv)
        last_conv = SAGEConv(h_feats, output_feats, 'mean')
        self.convs.append(last_conv)
        self.dropout = dropout
        self.skip = skip

    def forward(self, g, in_feat):
        prev_h = None
        h = self.convs[0](g, in_feat)
        h = F.relu(h) 
        h = F.dropout(h, p=self.dropout, training=self.training)
        for conv in self.convs[1:self.n_layers-1]:
            prev_h = h
            h = conv(g, h)            
            if self.skip:
                h = h + prev_h
            h = F.relu(h)
            h = F.dropout(h, p=self.dropout, training=self.training)
        h = self.convs[-1](g, h)
        return h

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import dgl.function as fn

class MLPUpgradedPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.
        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.
        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        avg = (edges.src['h']+edges.dst['h'])/2
        var = (edges.src['h']-edges.dst['h'])**2
        h = torch.cat([avg, var], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [ ]:
import torch 
import torch.nn.functional as F 

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def train(model, pred, train_g, train_pos_g, train_neg_g, optimizer, num_epochs=100):
    model.train()
    pred.train()
    for e in range(num_epochs):
        # forward
        h = model(train_g, train_g.ndata['feat'])
        pos_score = pred(train_pos_g, h)
        neg_score = pred(train_neg_g, h)
        loss = compute_loss(pos_score, neg_score)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {}'.format(e, loss))

### Prediction

In [ ]:
import torch
from sklearn.metrics import roc_auc_score 

def evaluate_torch(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
import dgl
import torch
import itertools
import numpy as np
import scipy.sparse as sp
import dgl.data
import networkx as nx

if __name__ == "__main__":
    #create dgl graph
    train_set = load_set(train=True)
    nx_g = set_to_directed_nx(train_set)
    node_dict = info_to_dict()
    nx.set_node_attributes(nx_g, values=node_dict, name="feat")
    sorted_nodes = sorted(nx_g.nodes())
    #creating convert dict to keep node embeddings
    convert_dict = {}
    for i, node in enumerate(sorted_nodes):
        convert_dict[node] = i
    g = dgl.from_networkx(nx_g, node_attrs=["feat"], idtype=torch.int32)

    #get positive attributes
    u, v = g.edges()
    eval_model = False
    if eval_model:
        eids = np.arange(g.number_of_edges())
        eids = np.random.permutation(eids)
        test_size = int(len(eids) * 0.1)
        train_size = g.number_of_edges() - test_size
        test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
        train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

        # Find all known negative edges and split them for training and testing
        non_edges = get_non_edges(train_set)
        non_edges_indic = np.ones((g.number_of_nodes(), g.number_of_nodes()))
        for elem in non_edges: 
            idx_0 = convert_dict[elem[0]] 
            idx_1 = convert_dict[elem[1]]
            non_edges_indic[idx_0][idx_1] = 0
            non_edges_indic[idx_1][idx_0] = 0
        neg_u, neg_v = np.where(non_edges_indic == 0)
        neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
        test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
        train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]
        train_g = dgl.remove_edges(g, eids[:test_size])
            
        #create train and test graphs
        train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
        train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
        test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
        test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

        #Define model, optimizer, and training step
        model = GraphSAGE(train_g.ndata['feat'].shape[1], 64, 32, n_layers=3, dropout=0.2, skip=True)
        pred = MLPUpgradedPredictor(32)
        optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.005, weight_decay=5*10**-4)
        train(model, pred, train_g, train_pos_g, train_neg_g, optimizer, num_epochs=250)

        #make a test
        with torch.no_grad():
            model.eval()
            pred.eval()
            h = model(train_g, train_g.ndata['feat'])
            pos_score = pred(test_pos_g, h)
            neg_score = pred(test_neg_g, h)
            print('AUC', evaluate_torch(pos_score, neg_score))

    else: 
        train_pos_g = dgl.graph((u, v), num_nodes=g.number_of_nodes())
        non_edges = get_non_edges(train_set)
        non_edges_indic = np.ones((g.number_of_nodes(), g.number_of_nodes()))
        for elem in non_edges: 
            idx_0 = convert_dict[elem[0]] 
            idx_1 = convert_dict[elem[1]]
            non_edges_indic[idx_0][idx_1] = 0
            non_edges_indic[idx_1][idx_0] = 0
        neg_u, neg_v = np.where(non_edges_indic == 0)
        train_neg_g = dgl.graph((neg_u, neg_v), num_nodes=g.number_of_nodes())
        model = GraphSAGE(g.ndata['feat'].shape[1], 32, 16, n_layers=3, dropout=0.2, skip=True)
        pred = MLPUpgradedPredictor(16)
        optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001)
        train(model, pred, g, train_pos_g, train_neg_g, optimizer, num_epochs=250)

        #make a test
        with torch.no_grad():
            model.eval()
            pred.eval()
            h = model(g, g.ndata['feat'])

        #get submission
        test_set = load_set(train=False)
        test_u = np.zeros(len(test_set))
        test_v = np.zeros(len(test_set))
        for i in range(len(test_set)):
            test_u[i] = convert_dict[test_set[i][0]]
            test_v[i] = convert_dict[test_set[i][1]]
        test_g = dgl.graph((test_u, test_v), num_nodes=g.number_of_nodes())
        with torch.no_grad():
            test_score = pred(test_g, h)
        test_pred = np.zeros(test_score.shape[0])
        for i, elem in enumerate(test_score):
            if elem < 0:
                test_pred[i] = 0
            else:
                test_pred[i] = 1
        n_test = len(test_set)
        create_submission(n_test, test_pred, pred_name="submissions/gnn")

In epoch 0, loss: 0.6832396984100342
In epoch 5, loss: 0.666217029094696
In epoch 10, loss: 0.6527720093727112
In epoch 15, loss: 0.6400973796844482
In epoch 20, loss: 0.6326701641082764
In epoch 25, loss: 0.6260826587677002
In epoch 30, loss: 0.614692211151123
In epoch 35, loss: 0.6078561544418335
In epoch 40, loss: 0.5978363156318665
In epoch 45, loss: 0.585607647895813
In epoch 50, loss: 0.5701477527618408
In epoch 55, loss: 0.5505990386009216
In epoch 60, loss: 0.530487596988678
In epoch 65, loss: 0.503662109375
In epoch 70, loss: 0.4856695234775543
In epoch 75, loss: 0.4582064747810364
In epoch 80, loss: 0.437043696641922
In epoch 85, loss: 0.4263246953487396
In epoch 90, loss: 0.405521422624588
In epoch 95, loss: 0.401401162147522
In epoch 100, loss: 0.38530537486076355
In epoch 105, loss: 0.36958053708076477
In epoch 110, loss: 0.3479953110218048
In epoch 115, loss: 0.3397076427936554
In epoch 120, loss: 0.3389747142791748
In epoch 125, loss: 0.3325619101524353
In epoch 130, los

## Use of Graph Clustering to add information

### Idea

A last predictor that was explored is a simple one, that still gives information: use of graph clustering methods in order to identify communities.

The link prediction happens the following way: if the two nodes involved are part of the same community, say there will be an edge. Otherwise, say there won't be an edge.

While this method is naïve, it increases the final score when paired with other methods. What happens is that it does not predict many edges in the end, but it can be expected that the ones that are predicted make sense.

The clustering method that was used was the Clauset-Newman-Moore modularity maximization to find the community partition with the largest modularity, that can be found implemented in networkx.

In [ ]:
import numpy as np
import networkx as nx 

from networkx.algorithms.community import greedy_modularity_communities

def get_community_based_pred(g, test_set):
    c = greedy_modularity_communities(g)
    comm_dict = get_community_dict(g, c)
    comm_pred = get_pred(comm_dict, test_set)
    return comm_pred

def get_community_dict(g, c):
    comm_dict = {}
    for i, community in enumerate(c):  
        for node in g.nodes: 
            if node in community:
                comm_dict[node] = i
    return comm_dict

def get_pred(comm_dict, test_set):
    comm_pred = np.zeros(len(test_set))
    for i, elem in enumerate(test_set):
        src, tgt = elem[0], elem[1]
        comm_src = comm_dict[src]
        comm_tgt = comm_dict[tgt]
        if comm_src == comm_tgt:
            comm_pred[i] = 1
    return comm_pred

### Prediction

In [ ]:
train_set = load_set(train=True)
g = set_to_nx(train_set)
test_set = load_set(train=False)
n_test = len(test_set)
community_pred = get_community_based_pred(g, test_set)
create_submission(n_test, community_pred, pred_name="submissions/community_pred")

## Final Prediction : Majority Vote

### Idea

The idea to make the final submissions was to exploit all information provided by the different methods that were tested:
- Supervised learning prediction performs link prediction using several global features.
- Unsupervised prediction performs link prediction using several local features.
- Deep learning prediction performs link prediction using node information.
- Graph clustering performs some more advanced but less complete prediction using the graph structure.

Therefore, in order to predict whether or not a link between two nodes will happen, the four models make a prediction. If two models happen to tell that this edge will happen, then it is marked as true. This method is the one that scored best on the Kaggle competition.

### Prediction

In [ ]:
import pandas as pd 

unsup_pred = pd.read_csv("submissions/unsupervised_pred.csv").to_numpy()
gnn_pred = pd.read_csv("submissions/gnn.csv").to_numpy()
sup_pred = pd.read_csv("submissions/supervised_pred.csv").to_numpy() 
comm_pred = pd.read_csv("submissions/community_pred.csv").to_numpy()
final_pred = (0.25*sup_pred+0.25*unsup_pred+0.25*gnn_pred+0.25*comm_pred)[:, 1]
for i in range(len(final_pred)):
    if final_pred[i] >= 0.5:
        final_pred[i] = 1
    else:
        final_pred[i] = 0
n_test = len(final_pred)
create_submission(n_test, final_pred, "submissions/avgd")

## Other Methods that were tried but failed

In this section, we report several methods that were tried but did not yield in an increase of results. All those methods can still be found coded within the zip folder sent with the report.

### Supervised Link Prediction

Combining local features and global features in the supervised predictor did not yield better performances.

Node2vec and Word2vec node embeddings were not good predictors at all in this use case, no matter the tuning of hyperparameters that was done.

Putting the node information into the predictors was not effective and reduced performance, no matter how it was put. We have tried:
- putting the full information vector into the classifier
-  putting a reduced information vector, whether it was through PCA or Embedded through a Graph Neural Network trained for Link Prediction like in the "Graph Neural Network" section
- using cosine similary on the information vectors in order to have symmetrical information

None of those methods resulted in an increase in performance.

### Graph Neural Networks based-approach

Using Simply a DotProduct predictor instead of a Neural Network resulted in far worse predictions.

VGNAE was tried as well: the idea was to mimic autoencoders for Graphs and recreate the adjacency matrix through latent space embedding, and use that to make predictions. While the method is extremely performant on datasets like Cora or Pubmed, it seems to extremely underperform in our use case. This may have been due to difficulties in hyperparameter tuning, or due to the fact the graph we have through the train.txt is not fit for this type of network.



### Clustering-based approach

Spectral Clustering was tried to perform link prediction like we've done with modularity maximization, but it required a too high k value to create significative communities. Even then, the communities it created were less significant than the communities created through modularity maximization. Therefore, we have dropped this approach.

## Conclusion

Overall, this competition was a great opportunity to experiment on graph-based link prediction.

## References

Graph features taxonomy: https://mdpi-res.com/d_attachment/make/make-02-00036/article_deploy/make-02-00036-v2.pdf?version=1608552901

Idea on how to use GraphSage for feature extraction was taken from: https://medium.com/stanford-cs224w/gnn-based-link-prediction-in-drug-drug-interaction-networks-c0e2136e4a72

Idea to use VGNAE: https://arxiv.org/pdf/2108.08046.pdf